In [45]:
# import itertools
# import pandas as pd
# import numpy as np
# from scipy.special import gammaln
# import networkx as nx

# def bayesian_score(G, data):
#     """
#     Computes the Bayesian score of the entire graph G given the data.
#     Args:
#         G: NetworkX DiGraph representing the Bayesian network structure.
#         data: pandas DataFrame with the observed data.
#     Returns:
#         The Bayesian score of the graph given the data.
#     """
#     total_score = 0

#     # Iterate over each node (variable) in the graph
#     for child in G.nodes():
#         ri = data[child].nunique()  # Number of unique states the child can take
#         parents = list(G.predecessors(child))  # Get the parents of the child node
#         if len(parents) == 0:  # If no parents, compute based on marginal probability
#             # No parents, so calculate for marginal distribution
#             for k in range(1, ri + 1):
#                 m_ijk = len(data[data[child] == k])
#                 total_score += gammaln(1 + m_ijk) - gammaln(1)  # Pseudocount = 1
#             total_score += gammaln(ri) - gammaln(ri + len(data))
#         else:
#             # If there are parents, calculate conditional distribution
#             # Get all unique parent configurations
#             parent_combinations = list(itertools.product(*[data[p].unique() for p in parents]))
            
#             for parent_inst in parent_combinations:
#                 # Filter data for this parent configuration
#                 subset = data.copy()
#                 for idx, parent in enumerate(parents):
#                     subset = subset[subset[parent] == parent_inst[idx]]
                
#                 # m_ij0: Total number of instances for this parent configuration
#                 m_ij0 = len(subset)
#                 total_score += gammaln(ri) - gammaln(ri + m_ij0)

#                 # Iterate over each possible state of the child
#                 for k in range(1, ri + 1):
#                     m_ijk = len(subset[subset[child] == k])  # Count of child being in state k
#                     total_score += gammaln(1 + m_ijk) - gammaln(1)  # Pseudocount = 1

#     return total_score

In [46]:
# import networkx as nx
# import pandas as pd

# # Function to parse the .gph file using NetworkX
# def parse_gph(file_path):
#     graph = nx.DiGraph()  # Directed graph for parent-child relationships
#     with open(file_path, 'r') as file:
#         for line in file:
#             parent, child = line.strip().split(',')
#             graph.add_edge(parent, child)  # Add directed edge from parent to child
#     return graph

# # Function to read the .csv file and return data as a pandas DataFrame
# def read_csv(file_path):
#     data = pd.read_csv(file_path)
#     return data

# # Example usage with your file paths
# csv_file = '/Users/tyang/repos/AA228-CS238-Student/project1/example/example.csv'
# gph_file = '/Users/tyang/repos/AA228-CS238-Student/project1/example/example.gph'

# # Load the data and graph
# data = read_csv(csv_file)
# graph = parse_gph(gph_file)

# # Compute the Bayesian score for the graph given the data
# score = bayesian_score(graph, data)
# print(f'Bayesian Score: {score}')

In [105]:
import itertools
import pandas as pd
import numpy as np
from scipy.special import gammaln
import networkx as nx

def bayesian_score(G, data):
    """
    Computes the Bayesian score of the entire graph G given the data.
    Args:
        G: NetworkX DiGraph representing the Bayesian network structure.
        data: pandas DataFrame with the observed data.
    Returns:
        The Bayesian score of the graph given the data.
    """
    total_score = 0

    # Iterate over each node (variable) in the graph
    for child in G.nodes():
        ri = max(data[child])  # Number of unique states the child can take, determined by the maximum value
        parents = list(G.predecessors(child))  # Get the parents of the child node
        
        if len(parents) == 0:  # If no parents, compute based on marginal probability
            # No parents, so calculate for marginal distribution
            for k in range(1, ri + 1):  # Iterate over each possible state of the child
                m_ijk = len(data[data[child] == k])  # Count of how many times child is in state k
                total_score += gammaln(1 + m_ijk) - gammaln(1)  # Pseudocount = 1 for uniform prior
            total_score += gammaln(ri) - gammaln(ri + len(data))
        
        else:
            # If there are parents, calculate conditional distribution
            # Get all unique parent configurations
            # parent_combinations = list(itertools.product(*[data[p].unique() for p in parents]))
            parent_combinations = list(itertools.product(*[range(1, max(data[p]) + 1) for p in parents]))

            for parent_inst in parent_combinations:
                # Filter data for this parent configuration
                subset = data.copy()
                for idx, parent in enumerate(parents):
                    subset = subset[subset[parent] == parent_inst[idx]]
                
                # m_ij0: Total number of instances for this parent configuration
                m_ij0 = len(subset)
                total_score += gammaln(ri) - gammaln(ri + m_ij0)

                # Iterate over each possible state of the child
                for k in range(1, ri + 1):
                    m_ijk = len(subset[subset[child] == k])  # Count of child being in state k
                    total_score += gammaln(1 + m_ijk) - gammaln(1)  # Pseudocount = 1

    return total_score

In [110]:
import networkx as nx
import pandas as pd

# Function to parse the .gph file using NetworkX
def parse_gph(file_path):
    graph = nx.DiGraph()  # Directed graph for parent-child relationships
    with open(file_path, 'r') as file:
        for line in file:
            parent, child = line.strip().split(',')
            graph.add_edge(parent, child)  # Add directed edge from parent to child
    return graph

# Function to read the .csv file and return data as a pandas DataFrame
def read_csv(file_path):
    data = pd.read_csv(file_path)
    return data

# Example usage with your file paths
# csv_file = '/Users/tyang/repos/AA228-CS238-Student/project1/example/example.csv'
# gph_file = '/Users/tyang/repos/AA228-CS238-Student/project1/example/example.gph'
csv_file = '/Users/tyang/repos/AA228-CS238-Student/project1/data/small.csv'
gph_file = '/Users/tyang/repos/AA228-CS238-Student/project1/result/small.gph'

# Load the data and graph
data = read_csv(csv_file)
graph = parse_gph(gph_file)

# Compute the Bayesian score for the graph given the data
score = bayesian_score(graph, data)
print(f'Bayesian Score: {score}')


Bayesian Score: -96911.23843553876


In [63]:
for child in graph.nodes():
    # print(child)
    pass

for parent in graph.predecessors("child1"):
    print(parent)
print(list(graph))

parent1
['parent1', 'child1', 'parent2', 'child2', 'parent3', 'child3']


In [82]:
# data[data["child1"] == 3]

In [86]:
for edge in graph.edges():
    print(edge[0], edge[1])

parent1 child1
parent1 child2
parent2 child2
parent3 child3
parent3 child2


In [87]:
# Function to save a .gph file from a NetworkX graph
def save_gph(G, file_path):
    with open(file_path, 'w') as f:
        for parent, child in G.edges():
            f.write(f'{parent},{child}\n')

save_gph(graph,"test.gph")

In [93]:
import networkx as nx
import matplotlib.pyplot as plt

# Function to visualize and save the graph as a .pdf file
def save_graph_as_pdf(G, file_path):
    plt.figure(figsize=(8, 6))  # Set the figure size
    pos = nx.spring_layout(G)  # Layout for positioning the nodes
    nx.draw(G, pos, with_labels=True, node_color='lightblue', node_size=2000, font_size=10, font_weight='bold', arrows=True)
    plt.title("Graph Visualization")
    plt.savefig(file_path, format='pdf')  # Save as .pdf file
    plt.close()  # Close the plot to avoid displaying

save_graph_as_pdf(graph, "test.pdf")

In [97]:

# Function to parse the .gph file
def parse_gph(file_path):
    graph = nx.DiGraph()  # Directed graph for parent-child relationships
    with open(file_path, 'r') as file:
        for line in file:
            parent, child = line.strip().split(',')
            graph.add_edge(parent, child)  # Add directed edge from parent to child
    return graph

# Function to visualize and save the graph as a .pdf file
def save_graph_as_pdf(G, file_path):
    plt.figure(figsize=(8, 6))  # Set the figure size
    pos = nx.spring_layout(G)  # Layout for positioning the nodes
    nx.draw(G, pos, with_labels=True, node_color='lightblue', node_size=2000, font_size=10, font_weight='bold', arrows=True)
    plt.title("Graph Visualization")
    plt.savefig(file_path, format='pdf')  # Save as .pdf file
    plt.close()  # Close the plot to avoid displaying

# Main function to parse the .gph file and save the graph as a .pdf file
def visualize_gph_as_pdf(gph_file_path, pdf_file_path):
    # Parse the .gph file
    G = parse_gph(gph_file_path)
    
    # Save the graph visualization as a .pdf file
    save_graph_as_pdf(G, pdf_file_path)

# Example usage
gph_file_path = '/Users/tyang/repos/AA228-CS238-Student/project1/example/example.gph'  # Path to the .gph file
pdf_file_path = 'test.pdf'  # Path to save the .pdf file

visualize_gph_as_pdf(gph_file_path, pdf_file_path)

print(f"Graph saved as {pdf_file_path}")

Graph saved as /Users/tyang/repos/AA228-CS238-Student/project1/example/example.pdf


In [104]:
from networkx.drawing.nx_pydot import write_dot
import os
import graphviz

def visualize_gph_with_graphviz(gph_file_path, dot_file_path, png_file_path):
    # Parse the .gph file
    G = parse_gph(gph_file_path)
    
    # Write the graph to a .dot file using NetworkX
    write_dot(G, dot_file_path)
    
    # Use GraphViz to convert the .dot file to a .png
    os.system(f"dot -Tpng {dot_file_path} -o {png_file_path}")
    print(f"Graph saved as {png_file_path}")

visualize_gph_with_graphviz("/Users/tyang/repos/AA228-CS238-Student/project1/example/example.gph", "/Users/tyang/repos/AA228-CS238-Student/project1/test.dot", "/Users/tyang/repos/AA228-CS238-Student/project1/test.png")


Graph saved as /Users/tyang/repos/AA228-CS238-Student/project1/test.png


In [111]:
import numpy as np
from scipy.special import gammaln

def bayesian_score(G, data, cache={}):
    """
    Computes the Bayesian score of the entire graph G given the data.
    Args:
        G: NetworkX DiGraph representing the Bayesian network structure.
        data: pandas DataFrame with the observed data.
        cache: Dictionary to store cached scores for node-parent configurations.
    Returns:
        The Bayesian score of the graph given the data.
    """
    total_score = 0

    # Convert DataFrame to NumPy array for efficient processing
    data_np = data.to_numpy()

    # Iterate over each node (variable) in the graph
    for child in G.nodes():
        parents = tuple(sorted(G.predecessors(child)))  # Get the parents of the child node, sorted to create a unique key
        
        # Create a unique cache key based on the child and its parents
        cache_key = (child, parents)

        if cache_key in cache:
            total_score += cache[cache_key]  # Use the cached score if available
        else:
            score = compute_local_bayesian_score_np(data, child, parents, data_np)
            cache[cache_key] = score  # Cache the score for future use
            total_score += score

    return total_score

def compute_local_bayesian_score_np(data, child, parents, data_np):
    """
    Computes the Bayesian score for a single node (child) and its parent set using NumPy for optimization.
    """
    total_score = 0
    child_idx = data.columns.get_loc(child)  # Get the index of the child in the DataFrame
    ri = np.max(data_np[:, child_idx])  # Number of unique states the child can take, determined by the maximum value

    if len(parents) == 0:  # If no parents, compute based on marginal probability
        # Vectorized computation for marginal distribution
        counts = np.bincount(data_np[:, child_idx], minlength=ri + 1)[1:]  # Counts for each state of the child (excluding 0)
        total_score += np.sum(gammaln(1 + counts) - gammaln(1))  # Pseudocount = 1
        total_score += gammaln(ri) - gammaln(ri + len(data_np))

    else:
        # Get the indices of the parents
        parent_idxs = [data.columns.get_loc(p) for p in parents]
        parent_combinations = np.array(list(itertools.product(*[range(1, np.max(data_np[:, idx]) + 1) for idx in parent_idxs])))

        for parent_inst in parent_combinations:
            # Vectorized filtering for parent configurations
            mask = np.all([data_np[:, parent_idxs[i]] == parent_inst[i] for i in range(len(parents))], axis=0)
            subset = data_np[mask]

            m_ij0 = len(subset)  # Total number of instances for this parent configuration
            total_score += gammaln(ri) - gammaln(ri + m_ij0)

            # Counts for each state of the child, given this parent configuration
            if len(subset) > 0:
                child_counts = np.bincount(subset[:, child_idx], minlength=ri + 1)[1:]  # Exclude 0
                total_score += np.sum(gammaln(1 + child_counts) - gammaln(1))

    return total_score

In [114]:
import networkx as nx
import pandas as pd

# Function to parse the .gph file using NetworkX
def parse_gph(file_path):
    graph = nx.DiGraph()  # Directed graph for parent-child relationships
    with open(file_path, 'r') as file:
        for line in file:
            parent, child = line.strip().split(',')
            graph.add_edge(parent, child)  # Add directed edge from parent to child
    return graph

# Function to read the .csv file and return data as a pandas DataFrame
def read_csv(file_path):
    data = pd.read_csv(file_path)
    return data

# Example usage with your file paths
csv_file = '/Users/tyang/repos/AA228-CS238-Student/project1/example/example.csv'
gph_file = '/Users/tyang/repos/AA228-CS238-Student/project1/example/example.gph'
# csv_file = '/Users/tyang/repos/AA228-CS238-Student/project1/data/medium.csv'
# gph_file = '/Users/tyang/repos/AA228-CS238-Student/project1/result/medium.gph'

# Load the data and graph
data = read_csv(csv_file)
graph = parse_gph(gph_file)

# Compute the Bayesian score for the graph given the data
score = bayesian_score(graph, data)
print(f'Bayesian Score: {score}')


Bayesian Score: -132.57689402451834
